In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
data = pd.read_csv("../data/Case3HistoricalPrices.csv")
data = data.drop(columns=['Unnamed: 0'])
data.head()

S1          S2          S3          S4          S5          S6  \
0  100.000000  100.000000  100.000000  100.000000  100.000000  100.000000   
1   98.355850   98.307641   95.630621   98.296988   98.474008   95.663069   
2   96.766305   97.785655   95.211317   97.876191   96.910785   95.299426   
3   99.498201   97.762932   95.831525   97.979482   99.992847   95.890271   
4   96.259221   92.942010   94.272932   92.958011   96.469517   94.318577   

           S7          S8          B1          B2  ...          B7  \
0  100.000000  100.000000  100.000000  100.000000  ...  100.000000   
1   98.342292   98.416837  100.086605   99.685232  ...   99.674428   
2   97.862363   96.850881  100.595828   99.614514  ...   99.463006   
3   97.890721   99.523488  100.687198   99.425275  ...   99.220046   
4   93.007052   96.278064  101.939638  100.099827  ...   99.843072   

           B8          C1          C2          C3          C4          C5  \
0  100.000000  100.000000  100.000000  100.000000  100.000000  100.000000   
1  100.131478   95.854652   95.571202   94.085760   93.956485   94.140624   
2  100.594669   87.500105   86.517650   88.861746   89.205952   89.255488   
3  100.639175   84.500885   83.306741   86.183137   86.617107   86.643905   
4  101.823484   79.974708   78.434282   80.585967   81.404902   81.484117   

           C6          C7          C8  
0  100.000000  100.000000  100.000000  
1   94.058865   95.786959   94.331430  
2   89.203200   86.873750   89.644765  
3   86.512592   83.788306   87.068357  
4   81.131945   79.157682   81.778226  

[5 rows x 24 columns]

In [11]:
# Initialize global variables
window = 90
bound = 1
window_prices = [data[col][:window].to_list() for col in data.columns]
moving_average = [sum(prices)/window for prices in window_prices]
moving_std = [np.std(prices) for prices in window_prices]

def allocate1(asset_prices):  
    lowest_asset = 0
    lowest_z = 100
    for i in range(len(asset_prices)):
        # Calculate z-score
        z = (asset_prices[i] - moving_average[i])/moving_std[i]
        if z < lowest_z:
            lowest_z = z
            lowest_asset = i
        # Update window_prices and moving_average
        window_prices[i].pop(0)
        window_prices[i].append(asset_prices[i])
        moving_average[i] = sum(window_prices[i])/window
        moving_std[i] = np.std(window_prices[i])
    portfolio = [0]*len(asset_prices)
    portfolio[lowest_asset] = 1
    return np.array(portfolio)

In [12]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = [0]*window

for index, row in data.iterrows():
    if index < window: # Skip training data
        continue
    if index == window: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate1(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate1(asset_prices)

In [13]:
total = 0
for i in range(1,11):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  -4.752828454657486
Sharpe ratio for year  2 :  -7.324673496702658
Sharpe ratio for year  3 :  -7.212911575911933
Sharpe ratio for year  4 :  -4.74190612047555
Sharpe ratio for year  5 :  -6.413875077466039
Sharpe ratio for year  6 :  -5.032720961927705
Sharpe ratio for year  7 :  -5.290257623097326
Sharpe ratio for year  8 :  -3.678849910858025
Sharpe ratio for year  9 :  -3.1271776239771722
Sharpe ratio for year  10 :  -2.756992060679874
Total points:  -50.33219290575377
